# DBX-Patch Quick Setup

This notebook installs and configures dbx-patch to enable editable package imports in Databricks.

**Run all cells in order.**


## Step 1: Install uv (Package Manager)


In [ ]:
# Install uv package manager
%pip install -q uv

# Verify installation
!uv --version

## Step 2: Install dbx-patch


In [ ]:
# Install dbx-patch library
# !uv pip install dbx-patch
!cd .. && uv build && uv pip install ./dist/dbx_patch-1.0.0-py3-none-any.whl

print("\n✓ dbx-patch installed successfully")

## Step 3: Apply All Patches


In [ ]:
# Import and apply all patches
from dbx_patch import patch_dbx

result = patch_dbx()

print("\n" + "=" * 80)
print("RESULTS:")
print(f"Overall success: {result.overall_success}")
print(f"Editable paths found: {len(result.editable_paths)}")

if result.editable_paths:
    print("\nEditable install paths:")
    for path in result.editable_paths:
        print(f"  - {path}")
else:
    print("\n⚠️  No editable installs detected yet.")
    print("   Install packages with: uv pip install -e /path/to/package")

## Step 4: Install sitecustomize.py (Optional but Recommended)

This makes patches apply automatically on kernel restart.


In [ ]:
# Install sitecustomize.py for automatic patching
from dbx_patch.install_sitecustomize import install_sitecustomize

result = install_sitecustomize()

if result:
    print("\n✓ sitecustomize.py installed!")
    print("  Patches will apply automatically on next kernel restart.")
else:
    print("\n⚠️  Installation failed")

## Step 5: Verify Installation


In [ ]:
from dbx_patch.patch_dbx import verify_editable_installs

result = verify_editable_installs()

print("\n" + "=" * 80)
print(f"Status: {result.status.upper()}")
print("=" * 80)

## Next Steps: Install Your Package

Now install your package as editable:


In [ ]:
# Example: Install your package as editable
# Replace with the path to your package

!uv pip install -e ../test_package

# Or if using uv.lock:
# !cd /Workspace/path/to/project && uv sync --active

print("Uncomment and modify the command above to install your package")

## Test Import

Try importing your package:


In [ ]:
# Enable logging at DEBUG level to see what's happening
import os

os.environ["DBX_PATCH_ENABLED"] = "1"
os.environ["DBX_PATCH_LOG_LEVEL"] = "DEBUG"

# Try importing your package
# Replace 'testx' with your package name
from test_package import hello

hello()

print("Uncomment and modify the import above to test your package")

In [ ]:
%restart_python

In [ ]:
from test_package import hello

hello()

## Troubleshooting

If imports still fail, run the diagnostic notebook:

```
notebooks/diagnostic_editable_imports.ipynb
```

Or check the solution guide:

```
docs/docs/files/solution-guide.md
```
